In [ ]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt

# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9, 3), np.float32)
objp[:,:2] = np.mgrid[0:9, 0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

#todo: set calib path
calib_path = "..."
image_names = glob.glob(calib_path + '*.jpg')
images = []

#read images
for i in range(0, len(image_names)):
    fname = image_names[i]
    img = cv2.imread(fname, -1)
    images.append(img)

print("loaded " + str(len(images)) + " images")

In [ ]:
#find the correspondences between object points and image points
imgpoints = [] # 2d points in image plane.
objpoints = []
for i in range(0, len(images)):
    img = images[i]
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Find the approximate chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6))

    #If found, add object points, image points (after refining them on subpixel lvl)
    if ret:
        print("found corners for img " + str(i))
        objpoints.append(objp)
        corners2 = cv2.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners2)

In [ ]:
for i in range(5, 6):
    fig = plt.figure(figsize=(30, 20))
    img = images[i]
    # Draw and display the corners
    plt.axis("off")
    img = cv2.cvtColor(cv2.drawChessboardCorners(img, (9, 6), imgpoints[i], True), cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    plt.show()

In [ ]:
shape = images[0].shape[:-1]

#find camera intrinsics and distortion parameters
imgpoints=np.stack(imgpoints)
_, intrinsics, dist_params, _, _ = cv2.calibrateCamera(objpoints, imgpoints, shape[::-1], None, None)

#get params necessary for undistortion: fx, fy, cx, cy, k1, k2, k3, p1, p2
fx = intrinsics[0, 0]
fy = intrinsics[1, 1]
cx = intrinsics[0, 2]
cy = intrinsics[1, 2]
k1 = dist_params[0, 0]
k2 = dist_params[0, 1]
k3 = dist_params[0, 4]
p1 = dist_params[0, 2]
p2 = dist_params[0, 3]

print("calibration finished")

In [ ]:
#create meshgrid of pixel coordinates
x = np.linspace(0, shape[1] - 1.0, shape[1])
y = np.linspace(0, shape[0] - 1.0, shape[0])
x, y = np.meshgrid(x, y)
#go to coordinates relative to image center and focal length (-c, / f)
x3d = (x - cx) / fx 
y3d = (y - cy) / fy
#print(np.min(x3d), np.max(x3d))
#precompute radiuses (r^2, r^4, r^6), xx, xy, yy
xx3d = x3d ** 2
yy3d = y3d ** 2
xy3d = x3d * y3d
r2 = xx3d + yy3d
r4 = r2 ** 2
r6 = r2 ** 3
#compute scale of radial distortion
scale = 1 + k1 * r2 + k2 * r4 + k3 * r6
#compute translation of tangential distortion
dx = 2 * p1 * xy3d + p2 * (r2 + 2 * xx3d)
dy = p1 * (r2 + 2 * yy3d) + 2 * p2 * xy3d
#undistort #(x, y) -> (x * scale + dx, y * scale + dy)
x3d_undist = x3d * scale + dx
y3d_undist = y3d * scale + dy
#map back to pixel coordinates (*f, +c)
xmap_undist = np.asarray(x3d_undist * fx + cx, dtype=np.float32)
ymap_undist = np.asarray(y3d_undist * fy + cy, dtype=np.float32)

print("undistortion mapping computed")

In [ ]:
#load a very distorted image
#map image via cv2.remap(img, mapx, mapy, cv2.INTER_LINEAR)
#compare to the mapping computed by opencv: mapx,mapy = cv2.initUndistortRectifyMap(mtx, dist, None, None, (4000, 2250), cv2.CV_32FC1)
#todo: set test img path
dist_test = "..."
fig = plt.figure(figsize=(30, 20))
img = images[0] #cv2.imread(dist_test, -1)
plt.axis("off")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img)
plt.show()
fig = plt.figure(figsize=(30, 20))
img = images[0]
plt.axis("off")
img = cv2.cvtColor(cv2.remap(img, xmap_undist, ymap_undist, cv2.INTER_LINEAR), cv2.COLOR_BGR2RGB)
plt.imshow(img)
plt.show()
mapx, mapy = cv2.initUndistortRectifyMap(intrinsics, dist_params, None, intrinsics, (4000, 2250), cv2.CV_32FC1)
print("max remapping error x:" + str(np.max(np.abs(xmap_undist - mapx))))
print("max remapping error y:" + str(np.max(np.abs(ymap_undist - mapy))))